In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.models as models
from torchvision import datasets, models, transforms

import os
import re
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image, ImageFile
from tqdm import tqdm
from skimage import io, transform
from skimage.util import img_as_float
from easydict import EasyDict as edict

from early_stopping_class import EarlyStopping
from data_loader import *

ImageFile.LOAD_TRUNCATED_IMAGES = True

__C = edict()
cfg = __C
__C.CUDA = torch.cuda.is_available()
__C.WORKERS = 4
__C.MAX_EPOCH = 100
__C.MIN_EPOCH = 5
__C.BATCH_SIZE = 32
__C.VERBOSE = True

# Fix the random seed
# torch.manual_seed(0)
# np.random.seed(0) 

# Check device, using gpu 0 if gpu exist else using cpu
device = torch.device("cuda:0" if cfg.CUDA else "cpu")
print(device)

In [ ]:
data_dir = '/'

train_loader, valid_loader = get_train_valid_loader(data_dir,
                                                    batch_size=32,
                                                    image_size=(224, 224),
                                                    augment=False,
                                                    random_seed=1993,
                                                    valid_size=0.2,
                                                    shuffle=True,
                                                    show_sample=True,
                                                    num_workers=cfg.WORKERS)

In [ ]:
def train_model(model, loss_fn, optimizer, train_generator, dev_generator, patience, n_epochs):
    """
    Perform the actual training of the model based on the train and dev sets.
    :param model: one of your models, to be trained to perform 4-way emotion classification
    :param loss_fn: a function that can calculate loss between the predicted and gold labels
    :param optimizer: a created optimizer you will use to update your model weights
    :param train_generator: a DataLoader that provides batches of the training set
    :param dev_generator: a DataLoader that provides batches of the development set
    :return model, the trained model
    """

    valloss = []
    valacc = []
    time0 = time.time()
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=cfg.VERBOSE)
    
    for epoch in range(n_epochs):
        start = time.time()
        epoch_loss = 0
        train_acc = 0
        j = 0
        model.train()
        for X_b, y_b in train_generator:
            j+=1
            num_b = len(y_b)
            X_b = X_b.float().to(device)          
            y_b = y_b.to(device)
            optimizer.zero_grad()
            y_pred = model(X_b)
            train_loss = loss_fn(y_pred, y_b.long())
            _,indices = y_pred.max(1)
            correct = (indices == y_b).float() #convert into float for division 
            train_acc += correct.sum() / len(correct)
            epoch_loss += train_loss
            train_loss.backward()
            optimizer.step()
        epoch_loss = epoch_loss/len(train_generator)
        train_acc = train_acc/len(train_generator)

    # on validation set
        model.eval()
        with torch.no_grad():
            valid_acc = 0
            valid_loss = 0
            for X_b, y_b in dev_generator:
                # Predict
                X_b = X_b.float().to(device)
                y_b = y_b.to(device)
                y_pred = model(X_b)
                valid_loss += loss_fn(y_pred, y_b.long()).item()
                _,indices = y_pred.max(1)
                correct = (indices == y_b).float()
                valid_acc += correct.sum() / len(correct)
            valid_loss = valid_loss/len(dev_generator)
            valid_acc = valid_acc/len(dev_generator)
            end = time.time()
            
            if cfg.VERBOSE:
                print(f'Epoch: {epoch+1:02}')
                print(f'\tTrain Loss: {epoch_loss:.3f}| Train Acc: {train_acc*100:.2f}% | Time: {end-start:.2f}s')
                print(f'\t Val. Loss: {valid_loss:.3f}|  Val. Acc: {valid_acc*100:.2f}%')
            
            valacc.append(valid_acc)
            valloss.append(valid_loss)
            
        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
    print('Total time = %.2fs'%(end - time0))
    return model,valloss

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.drop1 = nn.Dropout2d(p=0.2)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.drop2 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.drop1(x)
        x = x.view(-1, 16 * 53 * 53)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.softmax(x)
        return x

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# Flag for feature extracting. When False, we finetune the whole model, when True we only update the reshaped layer params
feature_extract = True

# model_ft = models.alexnet(pretrained=True)
# set_parameter_requires_grad(model_ft, feature_extract)
# num_ftrs = model_ft.classifier[6].in_features
# model_ft.classifier[6] = nn.Linear(num_ftrs, 3)

# model_ft = models.resnet18(pretrained=True)
# set_parameter_requires_grad(model_ft, feature_extract)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 3)

model_ft = models.vgg16_bn(pretrained=True)
set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, 512)

# print(model_ft)

In [ ]:
model = nn.Sequential(model_ft,
                      nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(inplace=True),
                      nn.Linear(256, 128), nn.ReLU(inplace=True),
                      nn.Linear(128, 3), nn.Softmax())
# print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001, amsgrad=False)

if torch.cuda.is_available():
        model.to(device)
        
dmodel,_ = train_model(model, loss_fn, optimizer, train_loader, valid_loader, patience=10, n_epochs=256)

In [ ]:
data_dir = '/home/jupyter/test_224/'

test_loader = get_test_loader(data_dir,
                            batch_size=32,
                            image_size=(224,224),
                            shuffle=True,
                            show_sample=True,
                            num_workers=4)

In [ ]:
predicted = []
img_name = []

dmodel.eval()

with torch.no_grad():
    for temp_data in test_loader:
        # Predict
        X_b = temp_data['image'].float().to(device)
        x_name = temp_data['name']
        y_pred = dmodel(X_b)
#         y_pred = F.softmax(y_pred)
        
        predicted.extend(y_pred.cpu().detach().numpy())
        img_name.extend(x_name)

In [ ]:
healthy_wheat = pd.Series(range(610), name="healthy_wheat", dtype=np.float32)
leaf_rust = pd.Series(range(610), name="leaf_rust", dtype=np.float32)
stem_rust = pd.Series(range(610), name="stem_rust", dtype=np.float32)

sub = pd.concat([healthy_wheat, leaf_rust, stem_rust], axis=1)

# append real predictions to the dataset
for i in tqdm(range(0 ,len(predicted))):
    sub.loc[i] = predicted[i]

sub["ID"] = img_name
cols = sub.columns.tolist()
cols = cols[-1:] + cols[:-1]
sub = sub[cols]
sub.to_csv("sub.csv", index=False)